# Check and run carbon cycle calibrations

From FaIR 2.0 from Nick. Calibrated on CMIP6 1pct runs. Let's see if they give reasonable concentrations in emissions-driven mode.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as pl
from climateforcing.utils import mkdir_p

from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, FAIR
from fair21.defaults import species_config_from_default

In [ ]:
co2_to_c = 44.009 / 12.011
co2_to_c

In [ ]:
# Source: Leach et al. 2021

# NB: rC and rA are in GtC units, we need to convert to GtCO2
data = np.array(
    [
        [36.73854601035055,25.589821019851797,40.704695982343765,38.09182601398885,35.70573492682388,34.26732262345922,32.223599635483424,33.39478016647172,33.33937342916488,40.735872526405046,37.91594456570033],
        [0.0349535801301073,0.00597614250950862,0.010664893971021883,0.0005810081769186404,-0.005958784801017192,0.021861410870304354,0.016608701817126814,0.013104461258272693,0.031043773610946346,0.009471296196005063,0.020138272127751655],
        [3.036651884848311,5.196160258410032,1.2786398011433562,2.472206604249436,-0.10385375927186047,4.855081881723322,1.0693983052255476,3.4644393974775767,1.499323874009292,1.5631932779473914,2.6714005898495543],
        [-0.0006603263192310749,0.004393751681079472,0.004211308668836011,0.009783189726962682,0.018116906645659014,-0.004242277713558451,0.012336113500092338,0.003993779169272571,-0.002570300844565665,0.004887468785878646,0.0018119017134572424]   
    ]
)
data[1,:] = data[1,:] / co2_to_c
data[3,:] = data[3,:] / co2_to_c

In [ ]:
params = pd.DataFrame(data.T, columns=['r0','rC','rT','rA'], index=[
    'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-ESM2-1', 'CanESM5', 'GFDL-ESM4', 'IPSL-CM6A-LR', 'MIROC-ES2L',
    'MPI-ESM1-2-LR', 'NorESM2-LM', 'UKESM1-0-LL'
])

In [ ]:
params.mean()

In [ ]:
params.corr()

In [ ]:
pd.plotting.scatter_matrix(params);

In [ ]:
samples = 1000000

kde = scipy.stats.gaussian_kde(params.T)
cc_sample = kde.resample(size=int(samples), seed=2421911)

mask = np.all(np.isnan(cc_sample), axis=0)
cc_sample = cc_sample[:,~mask]
cc_sample_df=pd.DataFrame(
    data=cc_sample[:,:samples].T, columns=['r0','rC','rT','rA']
)
#geoff_sample_df.to_csv('../data_output/geoff_sample.csv', index=False)
cc_sample_df

In [ ]:
pl.hist(cc_sample_df['r0']);

In [ ]:
pl.hist(cc_sample_df['rC']);

In [ ]:
pl.hist(cc_sample_df['rT']);

In [ ]:
pl.hist(cc_sample_df['rA']);

## First thing we'll do is run 11 ESM simulations, CO2 only, default forcing etc.

The CO2 concentrations and warming from these simulations will not be perfect because of the absence of other forcers. And the individual model values of CO2 ERF are not taken into account either.

In [ ]:
species_ids = {
    'co2_ffi': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'co2': SpeciesID('CO2', Category.CO2),
    'co2_afolu': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'ch4': SpeciesID('CH4', Category.CH4),
    'n2o': SpeciesID('N2O', Category.N2O),
}

In [ ]:
emitted_species = ['CO2_FFI', 'CO2_AFOLU']
from_other_species = ['co2']
dummy_species = ['ch4', 'n2o']
species_to_include = emitted_species + from_other_species + dummy_species
scenarios_to_include = ['ssp119', 'ssp245', 'ssp585']
scenarios = []

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        if species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2500'
        ].interpolate(axis=1).values.squeeze()
        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species.lower()], emissions=emis_in))
        
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    # add dummy species, as zero
    for species in dummy_species:
        list_of_species.append(Species(species_ids[species], emissions=np.zeros(751)))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
co2_conc_rcmip = {}
for scenario in scenarios_to_include:
    co2_conc_rcmip[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CO2")) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models_runs = {
    'ACCESS-ESM1-5': 'r1i1p1f1',
    'BCC-CSM2-MR': 'r1i1p1f1',
    'CESM2': 'r1i1p1f1',
    'CNRM-ESM2-1': 'r1i1p1f2',
    'CanESM5': 'r1i1p2f1', 
    'GFDL-ESM4': 'r1i1p1f1',
    'IPSL-CM6A-LR': 'r1i1p1f1',
    'MIROC-ES2L': 'r1i1p1f2',
    'MPI-ESM1-2-LR': 'r1i1p1f1',
    'NorESM2-LM': 'r1i1p1f1',
    'UKESM1-0-LL': 'r1i1p1f2',
}

configs = []

for imod, model in enumerate(models_runs):
    run = models_runs[model]
    condition = (df['model']==model) & (df['run']==run)
    climate_response = ClimateResponse(
        ocean_heat_capacity = df.loc[condition, 'C1':'C3'].values.squeeze(),
        ocean_heat_transfer = df.loc[condition, 'kappa1':'kappa3'].values.squeeze(),
        deep_ocean_efficacy = df.loc[condition, 'epsilon'].values[0],
        gamma_autocorrelation = df.loc[condition, 'gamma'].values[0],
        stochastic_run = False,
    )
    species_config = [species_config_from_default(species) for species in species_to_include]
    # CO2 is index 2
    species_config[2].iirf_0 = params.loc[model, 'r0']
    species_config[2].iirf_cumulative = params.loc[model, 'rC']
    species_config[2].iirf_airborne = params.loc[model, 'rA']
    species_config[2].iirf_temperature = params.loc[model, 'rT']
    
    configs.append(Config(model, climate_response, species_config))

In [ ]:
fair = FAIR(scenarios, configs, time=np.arange(1750.5, 2501))
fair.run()

In [ ]:
fair.temperature.shape

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.temperature[:, 2, :, 0, 0]);

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.concentration_array[:, 2, :, 2, 0]);
pl.plot(np.arange(1750.5, 2501), co2_conc_rcmip['ssp585'], color='k');

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.temperature[:, 1, :, 0, 0]);

In [ ]:
pl.plot(np.arange(1750.5, 2023), fair.temperature[:273, 1, :, 0, 0]);

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.concentration_array[:, 1, :, 2, 0]);
pl.plot(np.arange(1750.5, 2501), co2_conc_rcmip['ssp245'], color='k');

In [ ]:
pl.plot(np.arange(1750.5, 2023), fair.concentration_array[:273, 1, :, 2, 0]);
pl.plot(np.arange(1750.5, 2023), co2_conc_rcmip['ssp245'][:273], color='k');

In [ ]:
co2_conc_rcmip['ssp245'][264]

In [ ]:
fair.concentration_array[264, 1, :, 2, 0]
fair.concentration_array[264, 1, :, 2, 0].mean()

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.temperature[:, 0, :, 0, 0]);

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.concentration_array[:, 0, :, 2, 0]);
pl.plot(np.arange(1750.5, 2501), co2_conc_rcmip['ssp119'], color='k');

In [ ]:
fair.concentration_array[264, 1, :, 2, 0]  # year 2014

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.airborne_emissions_array[:, 1, :, 2, 0]/fair.cumulative_emissions_array[:,1,:,2,0]);

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.airborne_emissions_array[:, 2, :, 2, 0]/fair.cumulative_emissions_array[:,2,:,2,0]);

In [ ]:
pl.plot(np.arange(1750.5, 2501), fair.airborne_emissions_array[:, 0, :, 2, 0]/fair.cumulative_emissions_array[:,0,:,2,0]);

In [ ]:
mkdir_p('../data/parameter_sets/')

In [ ]:
cc_sample_df.to_csv('../data/parameter_sets/carbon_cycle.csv', index=False)